In [21]:
import notebook_common
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
)

p("Import common module")

Import common module


In [22]:
df_spoken = pd.read_csv(
    w_spoken_freq_nlb_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=["headword", "reading"],
)

df_written = pd.read_csv(
    w_written_freq_nlt_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=["headword", "reading"],
)

df_w_word_2400_without_guide_csv = pd.read_csv(
    w_word_2400_without_guide_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=0,
)
p("-" * 50 + "df_spoken" + "-" * 50)
df_spoken

--------------------------------------------------df_spoken--------------------------------------------------


,,spoken_freq
headword,reading,
の,ノ,57221425
に,ニ,35286930
を,ヲ,33707106
は,ハ,31279365
が,ガ,30705135
...,...,...
遡求する,ソキュウスル,1
遠駆け,トオガケ,1
遠近する,オチコチスル,1


In [23]:
p("-" * 50 + "df_written" + "-" * 50)
df_written

--------------------------------------------------df_written--------------------------------------------------


,,written_freq
headword,reading,
いる-非自立,イル,968064
こと,コト,701470
の,ノ,686025
する,スル,607405
ある,アル,496737
...,...,...
所思,ショシ,1
襲用,シュウヨウ,1
襲爵,シュウシャク,1


In [24]:
p("-" * 50 + "df_w_word_2400_without_guide_csv" + "-" * 50)
df_w_word_2400_without_guide_csv

--------------------------------------------------df_w_word_2400_without_guide_csv--------------------------------------------------


,word,chinese,property,meaning,new_chinese,word_repetition
no,,,,,,
1,ああ,NaN,感,啊，嗯；啊呀，唉。,ああ,1
2,あい,愛,名・动３他,爱情；爱好。,愛,1
3,あいさつ,NaN,名・动３自,寒暄；致辞。,あいさつ,1
4,あいず,合図,名,信号。,合図,1
5,アイスクリーム,NaN,名,冰激凌；雪糕。,アイスクリーム,1
...,...,...,...,...,...,...
2415,わる,割る,动1他,"切,割;把一物分成几部分;除",割る,1
2416,わるい,悪い,形1,"坏;恶性;不对,错误",悪い,1
2417,われる,割れる,动2自,破碎;裂开,割れる,1


In [25]:
ids_df_spoken = set(df_spoken.index)
ids_df_written = set(df_written.index)
common_ids = ids_df_spoken & ids_df_written
print(len(common_ids))

77273


In [26]:
df_w_word_2400_without_guide_csv.index

Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
       ...
       2410, 2411, 2412, 2413, 2414, 2415, 2416, 2417, 2418, 2419],
      dtype='int64', name='no', length=2419)

In [27]:
df_merged = pd.concat([df_spoken, df_written], axis=1, join="outer")


def type_dection(spoken, written):
    if spoken > 0 and written > 0:
        return "both"
    elif np.isnan(spoken):
        return "written"
    elif np.isnan(written):
        return "spoken"
    else:
        return "empty"


df_merged["type"] = df_merged.apply(
    lambda x: type_dection(x["spoken_freq"], x["written_freq"]), axis=1)

# 查看headword重复的次数
df_merged["headword_repetition"] = df_merged.groupby(["headword"
                                                     ]).transform("size")

# 只对headword列进行统计
df_merged["gr_by_word_sum_s_freq"] = df_merged.groupby(
    ["headword"])["spoken_freq"].transform("sum")
df_merged["gr_by_word_sum_w_freq"] = df_merged.groupby(
    ["headword"])["written_freq"].transform("sum")

# df_merged.index
to_csv_sig(df_merged, w_merged_freq_sumed_csv, True)
p("w_merged_freq_sumed_csv--ok")

w_merged_freq_sumed_csv--ok


In [28]:
df_merged

,,spoken_freq,written_freq,type,headword_repetition,gr_by_word_sum_s_freq,gr_by_word_sum_w_freq
headword,reading,,,,,,
の,ノ,57221425.0,686025.0,both,1,57221425.0,686025.0
に,ニ,35286930.0,NaN,spoken,1,35286930.0,0.0
を,ヲ,33707106.0,NaN,spoken,1,33707106.0,0.0
は,ハ,31279365.0,NaN,spoken,1,31279365.0,0.0
が,ガ,30705135.0,NaN,spoken,1,30705135.0,0.0
...,...,...,...,...,...,...,...
8ミリビデオカメラ,ハチミリビデオカメラ,NaN,1.0,written,1,0.0,1.0
鼻緒する,ハナオスル,NaN,1.0,written,1,0.0,1.0
DJする,ディージェイスル,NaN,1.0,written,1,0.0,1.0


In [44]:
#找出2400价目词汇里有多少个在词频里
word2400_set = set(df_w_word_2400_without_guide_csv['new_chinese'])
p(len(word2400_set))

headword_freq_merge_set = set(df_merged.index.get_level_values(0))
p(len(headword_freq_merge_set))

reading_freq_merge_set = set(df_merged.index.get_level_values(1))
p(len(reading_freq_merge_set))

n = 0
effect = 0
for word in word2400_set:
    if word in headword_freq_merge_set or word in reading_freq_merge_set:
        effect += 1
    n += 1
p(effect / n)

2403
102381
80035
0.8959633791094466


In [ ]:
df_freq_2400 = df_deleted_word_removed.join(df_merged, how="left")
df_freq_2400
df.to_csv(export_2400, index=False, encoding="utf-8-sig")